In [ ]:
#conda install lightgbm

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import OneHotEncoder

In [3]:
# Load test data
test = pd.read_csv('songs_test.csv')
test = test.set_index('song_id')
test.head()

,audio_danceability,audio_energy,audio_key,audio_loudness,audio_mode,audio_speechiness,audio_acousticness,audio_instrumentalness,audio_liveness,audio_valence,...,lyrics_year,lyrics_years,lyrics_yellow,lyrics_yes,lyrics_yesterday,lyrics_yet,lyrics_yo,lyrics_york,lyrics_young,lyrics_zone
song_id,,,,,,,,,,,,,,,,,,,,,
10001,0.862,0.648,4,-7.401,0,0.2510,0.0455,0.000000,0.0332,0.637,...,1,0,0,0,0,0,0,0,0,0
10002,0.664,0.842,9,-5.351,1,0.3390,0.2080,0.000006,0.5540,0.719,...,0,0,0,0,0,0,1,0,0,0
10003,0.633,0.752,9,-7.227,1,0.0323,0.0542,0.001990,0.0432,0.762,...,0,0,0,1,0,0,0,0,0,0
10004,0.742,0.527,8,-6.892,0,0.0769,0.3270,0.000000,0.2500,0.638,...,0,0,0,0,0,0,0,0,0,0
10005,0.682,0.574,4,-5.241,0,0.0519,0.0355,0.000131,0.2540,0.731,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Load training data (labelled)
train = pd.read_csv('songs_train.csv')
train = train.set_index('song_id')
train.head()

,genre,audio_danceability,audio_energy,audio_key,audio_loudness,audio_mode,audio_speechiness,audio_acousticness,audio_instrumentalness,audio_liveness,...,lyrics_year,lyrics_years,lyrics_yellow,lyrics_yes,lyrics_yesterday,lyrics_yet,lyrics_yo,lyrics_york,lyrics_young,lyrics_zone
song_id,,,,,,,,,,,,,,,,,,,,,
1,pop,0.733,0.886,2,-1.856,0,0.0426,0.0481,0.000002,0.1910,...,0,0,0,0,0,0,0,0,0,0
2,pop,0.685,0.886,0,-4.179,1,0.0340,0.0280,0.000000,0.4810,...,0,0,0,0,0,0,0,0,0,0
3,pop,0.691,0.695,0,-5.600,1,0.0367,0.0592,0.000000,0.0647,...,0,0,0,0,0,0,0,0,0,0
4,pop,0.761,0.739,6,-3.591,1,0.0369,0.1780,0.000003,0.2300,...,0,0,0,0,0,0,0,0,0,0
5,pop,0.744,0.828,2,-5.911,1,0.0289,0.0646,0.000014,0.0858,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Integer encoding target variable
genres = {'pop': 0,
          'hip hop': 1,
          'rock': 2,
          'rap':3
         }
num2gen = {0 :'pop',
          1 :'hip hop',
          2 : 'rock',
          3 : 'rap'
         }

In [6]:
# Split train and test data
X = train.drop('genre', axis=1).copy()
y = train['genre'].copy().map(genres)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Base Model Light GBM

In [7]:
train_data=lgb.Dataset(X_train, label=y_train)

evals={}
callbacks = [lgb.record_evaluation(evals)]

param = {'objective':'multiclass', 'num_class': 4,'learning_rate':.01,'max_bin':200,
        'metric': ['multi_error', 'multi_logloss']};

lgb_clf = lgb.train(param, train_data, callbacks=callbacks)

lgb_pred = lgb_clf.predict(X_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035251 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11131
[LightGBM] [Info] Number of data points in the train set: 7500, number of used features: 1233
[LightGBM] [Info] Start training from score -1.368847
[LightGBM] [Info] Start training from score -1.463896
[LightGBM] [Info] Start training from score -1.309827
[LightGBM] [Info] Start training from score -1.408949


In [8]:
lgb_clf = lgb.LGBMClassifier(objective='multiclass', 
                            num_class=4,
                            metric= ['multi_error', 'multi_logloss'],
                            seed=42)

In [9]:
lgb_clf.fit(X_train, 
            y_train,
            verbose = 0,
            eval_set=[(X_train, y_train), (X_test, y_test)])

LGBMClassifier(metric=['multi_error', 'multi_logloss'], num_class=4,
               objective='multiclass', seed=42)

In [10]:
lgb_pred = lgb_clf.predict(X_test)

In [11]:
lgb_pred

array([2, 1, 2, ..., 2, 2, 0])

In [12]:
## ---------- Model Classification Report ----------
## get predictions and create model quality report


print('\n------------------ Confusion Matrix -----------------\n')
print(confusion_matrix(y_test, lgb_pred))

print('\n-------------------- Key Metrics --------------------')
print('\nAccuracy: {:.2f}'.format(accuracy_score(y_test, lgb_pred)))
print('Balanced Accuracy: {:.2f}\n'.format(balanced_accuracy_score(y_test, lgb_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, lgb_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, lgb_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, lgb_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, lgb_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, lgb_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, lgb_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, lgb_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, lgb_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, lgb_pred, average='weighted')))

print('\n--------------- Classification Report ---------------\n')
print(classification_report(y_test, lgb_pred))
print('---------------------- LIGHT G BOOST ----------------------') # unnecessary fancy styling


------------------ Confusion Matrix -----------------

[[490  30  85  52]
 [ 47 302  22 183]
 [ 95  21 554   2]
 [ 80 167  14 356]]

-------------------- Key Metrics --------------------

Accuracy: 0.68
Balanced Accuracy: 0.67

Micro Precision: 0.68
Micro Recall: 0.68
Micro F1-score: 0.68

Macro Precision: 0.67
Macro Recall: 0.67
Macro F1-score: 0.67

Weighted Precision: 0.68
Weighted Recall: 0.68
Weighted F1-score: 0.68

--------------- Classification Report ---------------

              precision    recall  f1-score   support

           0       0.69      0.75      0.72       657
           1       0.58      0.55      0.56       554
           2       0.82      0.82      0.82       672
           3       0.60      0.58      0.59       617

    accuracy                           0.68      2500
   macro avg       0.67      0.67      0.67      2500
weighted avg       0.68      0.68      0.68      2500

---------------------- LIGHT G BOOST ----------------------


In [13]:
# Make a prediction
LGB1_pred = pd.DataFrame(lgb_clf.predict(test)).set_index(test.index).rename(columns={0:'genre'})
LGB1_pred = LGB1_pred['genre'].map(num2gen)
LGB1_pred.to_csv('LGB1_prediction.csv')